In [3]:
import pandas as pd
from pathlib import Path

dir = Path('./full')

csvs = list(dir.glob('*.csv'))

dfs = []
for i, csv in enumerate(csvs):
    print(f'{i}/{len(csvs)}\t{csv}.')
    df = pd.read_csv(csv, header=None)
    if len(df.columns) == 5:
        df.columns = ['dn', 'dac_csv_id', 'ts_begin', 'ts_end', 'family']
    else:
        df.columns = ['dn', 'dac_csv_id', 'family']
        df['ts_begin'] = None
        df['ts_end'] = None
        pass
    dfs.append(df)
    pass

DF = pd.concat(dfs) # 3 minutes
DF = DF.reset_index(names='dac_original_index')

# total running of this cell: ~3m 30s [without sort 1.23s]

0/93	full/ekforward_dga.csv.
1/93	full/oderoor_dga.csv.
2/93	full/randomloader_dga.csv.
3/93	full/ramdo_dga.csv.
4/93	full/locky_dga.csv.
5/93	full/omexo_dga.csv.
6/93	full/goznym_dga.csv.
7/93	full/madmax_dga.csv.
8/93	full/sphinx_dga.csv.
9/93	full/diamondfox_dga.csv.
10/93	full/tinynuke_dga.csv.
11/93	full/necurs_dga.csv.
12/93	full/simda_dga.csv.
13/93	full/pitou_dga.csv.
14/93	full/ramnit_dga.csv.
15/93	full/makloader_dga.csv.
16/93	full/emotet_dga.csv.
17/93	full/xshellghost_dga.csv.
18/93	full/gameover_dga.csv.
19/93	full/mirai_dga.csv.
20/93	full/rovnix_dga.csv.
21/93	full/murofet_dga.csv.
22/93	full/qakbot_dga.csv.
23/93	full/modpack_dga.csv.
24/93	full/bedep_dga.csv.
25/93	full/conficker_dga.csv.
26/93	full/tinba_dga.csv.
27/93	full/szribi_dga.csv.
28/93	full/nymaim_dga.csv.
29/93	full/bamital_dga.csv.
30/93	full/pykspa2s_dga.csv.
31/93	full/tofsee_dga.csv.
32/93	full/volatilecedar_dga.csv.
33/93	full/chinad_dga.csv.
34/93	full/downloader_dga.csv.
35/93	full/ranbyus_dga.csv.


In [4]:

codes, uniques = DF['family'].factorize()

s = pd.Series(uniques).str.split('_dga_', expand=True)

s.columns = ['family', 'hash']

family_names = s['family'].factorize()
hash_names = s['hash'].factorize()

dac_malware_table = s.copy()
dac_malware_table.index += 1
dac_malware_table = dac_malware_table[['hash','family']].reset_index(names='id')
dac_malware_table.to_csv('dac_malware_table.csv', index=False)

DF['family'] = codes + 1
DF.rename(columns={'family': 'malware_id'}, inplace=True)


In [5]:
"""
Test for IDs
"""

_a, _b = dac_malware_table['id'].min(), DF['malware_id'].min()
if not (_a == _b):
    print('test1', f'Error, minimum id value should be one, instead is: {_a} and {_b}')
_a, _b = dac_malware_table['id'].max(), DF['malware_id'].max()
if not (_a == _b):
    print('test2', f'Error, maximum id value should be one, instead is: {_a} and {_b}')


In [6]:
"""
Saving all
"""

# 30 secondi
DF.to_pickle('DF.pickle')

DF.index += 1
# 3 minuti e 30 secondi
DF[['dn', 'malware_id', 'ts_begin', 'ts_end', 'dac_csv_id', 'dac_original_index']].reset_index(names='id').to_csv('DF.csv', index=False)

# DAC

## Create Tables query

```sql
DROP TABLE IF EXISTS public.dac;
DROP TABLE IF EXISTS public.dac_malware;

CREATE TABLE IF NOT EXISTS public.dac_malware
(
    id bigint NOT NULL,
    hash character(8) COLLATE pg_catalog."default",
    family text NOT NULL,
    CONSTRAINT dac_malware_pkey PRIMARY KEY (id)
)
TABLESPACE pg_default;

CREATE TABLE IF NOT EXISTS public.dac
(
    id bigint NOT NULL GENERATED ALWAYS AS IDENTITY ( INCREMENT 1 START 1 MINVALUE 1 MAXVALUE 9223372036854775807 CACHE 1 ),
    dn text COLLATE pg_catalog."default",
    malware_id bigint,
    ts_begin timestamp without time zone,
    ts_end timestamp without time zone,
    dac_csv_id int,
    dac_original_index bigint,
    CONSTRAINT dac_pkey PRIMARY KEY (id),
    CONSTRAINT dac_malware_fkey FOREIGN KEY (malware_id)
        REFERENCES public.dac_malware (id) MATCH SIMPLE
        ON UPDATE NO ACTION
        ON DELETE NO ACTION
)
TABLESPACE pg_default;

ALTER TABLE IF EXISTS public.dac
    OWNER to postgres;

COMMENT ON COLUMN public.dac.dac_csv_id
    IS 'The second column field present in the csv files.';

COMMENT ON COLUMN public.dac.dac_original_index
    IS 'The index in the original file of the 2020 dgarchive dataset.';

ALTER TABLE IF EXISTS public.dac_malware
    OWNER to postgres;
```


## COPY csv to tables

Notice: remember to substitute the paths.

## Upload *dac_malware* table:

```sql
COPY PUBLIC.DAC_MALWARE (ID, HASH, FAMILY)
FROM
	'[path-to:dac_malware_table.csv]' DELIMITER ',' CSV HEADER QUOTE '"' ESCAPE '''';
```

## Upload *dac* table:

```sql
COPY PUBLIC.DAC (
	ID,
	DN,
	MALWARE_ID,
	TS_BEGIN,
	TS_END,
	DAC_CSV_ID,
	DAC_ORIGINAL_INDEX
)
FROM
	'[path-to:DF.csv]' DELIMITER ',' CSV HEADER QUOTE '"' ESCAPE '''';
```

## Create *dac_dn_index* index:

```sql
CREATE INDEX IF NOT EXISTS dac_dn_index
    ON public.dac USING btree
    (dn COLLATE pg_catalog."default" ASC NULLS LAST)
    INCLUDE(dn, ts_begin, ts_end)
    WITH (deduplicate_items=True)
    TABLESPACE pg_default;
```



## UPDATE *dn* table setting *dac_id*

```sql
UPDATE DN
SET DN.dac_id = COALESCE(dac.id, NULL)
FROM dac
WHERE DN.DN = DAC.DN;

```